# Importing Modules

In [1]:
import numpy as np

# Defining Methods

In [1]:
class CFAR(object):
    
    def __init__(self,img,bw,gw,tw,pfa,kernel):
        self.bw = bw
        self.gw = gw
        self.tw = tw
        self.pfa = pfa
        self.kernel = kernel
        self.img = self.mean_filter(img)
        self.sf = self.scaleFactor()
        self.dvi_img = []
        self.p_img = []
        self.finalImg = []
        print("Kernel Ready...")

    # Getting sliding window
    #img is the image and m is the size of window (eg 3x3, 5x5)
    def Shipdetection(self):
        print("ShipDetection Procecss has started...")
             #make the loop for 3 and 5.
        s,k = self.img.shape
        for i in range(int(self.bw/2)):
            z = np.zeros([s,1])
            self.img = np.concatenate((z,self.img),axis=1)

            z = np.zeros([s,1])
            self.img = np.concatenate((self.img,z),axis=1)

            s = s+2

            z = np.zeros([1,s])
            self.img = np.concatenate((self.img,z),axis=0)

            z = np.zeros([1,s])
            self.img = np.concatenate((z,self.img),axis=0)    


        stepSize = 1
        w_width = self.bw
        w_height = w_width
        print("Computing DVI and noise...")
        for x in range(0,  self.img.shape[1] - w_width +1, stepSize):
            for y in range(0, self.img.shape[0] - w_height +1, stepSize):
                window= np.array(self.img[x:x + w_width, y:y + w_height])
                
                guard,noise = (self.get_GuardWindow(window))
                P = self.noisePower(noise)
                #P = 230
                
                target,back = self.get_TargetWindow(guard)
                Dvi = self.DetectionVariable(target,back)
                self.dvi_img.append(Dvi)

                T = self.sf*P
                self.p_img.append(T)

                if Dvi > T:
                    self.finalImg.append(0)
                else:
                    self.finalImg.append(1) #Valid Ships 
        
        self.finalImg = np.array(self.finalImg).reshape(k,k)
        print("Process Completed.")
        return self.finalImg

    
    
    def get_sf(self):
        return self.sf
    
    def get_dvImage(self):
        return self.dvi_img
    
    def get_pImage(self):
        return self.p_img
    
    #getting center_pixel for every sliding window
    def get_center_pixel(self,window):
        x_i = int(len(window)/2)
        return window[x_i][x_i]

    #getting mean of sliding window
    def getMean(self,window):
        return window.mean()

    #getting stdev of sliding window
    def getStdev(self,window):
        return window.std()  

    def DVmean(self,arr):
        s = 0.0
        for i in arr:
            s += i
        return (s/((self.gw**2)-(self.tw**2)))

    def DVstd(self,arr,mean):
        s = 0.0
        l = self.gw**2 - self.tw**2
        for i in range(l):
            if i > len(arr)-1:
                s += mean**2
            else:
                res = (arr[i] - mean)**2
                s += res
        return np.sqrt(s/(l-1))


    #computing threshold for the sliding window
    def DetectionVariable(self,window,arr):
        center_pixel = self.get_center_pixel(window)
        #print(center_pixel)
        win_mean = self.DVmean(arr)
        #win_mean = getMean(window)
        #print(win_mean)
        win_stdev = self.DVstd(arr,win_mean)
        #win_stdev = getStdev(window)
        #print(win_stdev)
        return (abs(center_pixel-win_mean)/win_stdev)

    #Computing the target window of size 3x3 and returning along with the
    #background pixels
    def get_TargetWindow(self,img):
        s = []
        i = -int(self.tw/2)
        #print(i)
        for _ in range(self.tw):
            #s = [img[int(bw/2)-1][int(bw/2)-1:int(bw/2)+2], img[int(bw/2)][int(bw/2)-1:int(bw/2)+2], img[int(bw/2)+1][int(bw/2)-1:int(bw/2)+2]]
            s.append(img[int(self.gw/2)+i][int(self.gw/2)-int(self.tw/2):int(self.gw/2)+int(self.tw/2)+1])
            i += 1

        s = (np.array(s))

        background = []
        for z in img:
            for t in z:
                if t in s:
                    pass
                else:
                    background.append(t)
        background = np.array(background)
        return [s,background]

    def get_GuardWindow(self,img):
        s = []
        i = -int(self.gw/2)
        #print(i)
        for _ in range(self.gw):
            #s = [img[int(bw/2)-1][int(bw/2)-1:int(bw/2)+2], img[int(bw/2)][int(bw/2)-1:int(bw/2)+2], img[int(bw/2)+1][int(bw/2)-1:int(bw/2)+2]]
            s.append(img[int(self.bw/2)+i][int(self.bw/2)-int(self.gw/2):int(self.bw/2)+int(self.gw/2)+1])
            i += 1

        s = (np.array(s))

        background = []
        for z in img:
            for t in z:
                if t in s:
                    pass
                else:
                    background.append(t)
        background = np.array(background)
        return [s,background]

    def noisePower(self,arr):
        s = 0.0
        for i in arr:
            s += i
        return (s/((self.bw**2)-(self.gw**2)))

    def scaleFactor(self):
        N = self.bw**2 - self.gw**2
        alpha = N*(self.pfa**(-1/N) -1)
        return alpha

    def mean_filter(self,img):

        s,k = img.shape
        for i in range(int(self.kernel/2)):
            z = np.zeros([s,1])
            img = np.concatenate((z,img),axis=1)

            z = np.zeros([s,1])
            img = np.concatenate((img,z),axis=1)

            s = s+2

            z = np.zeros([1,s])
            img = np.concatenate((img,z),axis=0)

            z = np.zeros([1,s])
            img = np.concatenate((z,img),axis=0)    

        
        mean_filter_img = []
        stepSize = 1
        w_width = self.kernel
        w_height = w_width
        for x in range(0,  img.shape[1] - w_width +1, stepSize):
            for y in range(0, img.shape[0] - w_height +1, stepSize):
                window = np.array(img[x:x + w_width, y:y + w_height])

                cal_mean = self.getMean(window)
                mean_filter_img.append(cal_mean)

        mean_filter_img = np.array(mean_filter_img).reshape(k,k)
        return mean_filter_img